# Fake news detection

#### Краткое описание работы

Чтобы векторизовать заголовок будет использована предобученная модель эмбеддингов от navec, как среднее эмбеддингов всех слов в заголовке для простоты(как я убедился ниже, это очень эффективное по затратам решение задачи с хорошим качеством)

Работа с незнакомыми словами:                            
1. для каждого слова из navec.vocab.words получаю n-граммы
2. каждой уникальной n-грамме ставлю в соотвествие среднее всех векторов в которых она содержится
3. если вижу незнакомое слово, то разбиваю его на n-граммы
4. усредняю n-граммы незнакомого слова(те, что встречались в n-граммах от navec)

Для работы с нейронными сетями выбрал pytorch

Использую 3 линейных слоя и функцию активации: гиперболический тангенс

функция потерь: CrossEntropyLoss

оптимизатор: Adam

Кол-во эпох: 10

Размер батча: 16

Примерное время обучения на трейне: 5сек

#### импорты

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam

import string

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lvg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### загрузка данных

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
#для коллаба
!pip install navec

In [5]:
#для коллаба
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [6]:
from navec import Navec

path_news = 'navec_news_v1_1B_250K_300d_100q.tar'  

In [7]:
navec_news = Navec.load(path_news)  

In [367]:
# для обучения
data = pd.read_csv('train.tsv', sep='\t')

# для предсказаний
make_predict = pd.read_csv('test.tsv', sep = '\t')

In [368]:
data.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


In [369]:
# разделим на train и test
train_data, test_data = train_test_split(data, shuffle = True, test_size=0.2, random_state=44)

# обновим идексы
train_data.reset_index(drop = True, inplace = True)
test_data.reset_index(drop = True, inplace = True)
 
#посморим на наши датасеты
print('train:')
print(train_data.describe())
print('test:')
print(test_data.describe())

train:
           is_fake
count  4606.000000
mean      0.499566
std       0.500054
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000
test:
           is_fake
count  1152.000000
mean      0.501736
std       0.500214
min       0.000000
25%       0.000000
50%       1.000000
75%       1.000000
max       1.000000


как видим, выборка сбалансирована, все хорошо)

#### посмотрим, каких слов не знает navec
(ради интереса)

In [370]:
tokenizer = nltk.WordPunctTokenizer()

In [371]:
# сколько стоп слов есть в датасете
stop_words_in_corpus = []

for title in data['title']:
    tokens = tokenizer.tokenize(title.lower())
    stop_tokens = [token for token in tokens if token in stopwords.words('russian')]
    for token_word in stop_tokens:
        stop_words_in_corpus.append(token_word)
        
len(set(stop_words_in_corpus))

123

In [372]:
# считаем кол-во незнакомых слов одновременно с фильтрацией
unknown_tokens = []
for title in data['title']:
    tokens = tokenizer.tokenize(title.lower())
    filtered_tokens = [token for token in tokens if (len(token) > 3) and (token not in stopwords.words('russian')) and token.isalpha()]
    for token_word in filtered_tokens:
        if token_word not in navec_news:
            unknown_tokens.append(token_word)

In [373]:
# кол-во незнакомых слоunknown_tokens
len(set(unknown_tokens))

1319

In [374]:
navec_news['<pad>']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [375]:
navec_news['<unk>']

array([ 3.69125791e-02,  9.32818875e-02,  2.01917738e-02, -7.86257535e-02,
       -1.01714201e-01,  1.51891438e-02,  9.91745573e-03,  4.18423414e-02,
       -6.39119446e-02, -1.38967847e-02, -2.78906524e-02,  4.35626879e-02,
        2.56296489e-02, -5.87309301e-02, -9.63660888e-03,  6.80894479e-02,
        1.83931947e-01, -9.04254615e-02,  7.05215931e-02, -1.12063460e-01,
        1.38030723e-01, -2.67519075e-02,  5.57659902e-02, -3.90229225e-02,
       -2.08702944e-02,  7.01430961e-02, -1.06053390e-02, -3.07631604e-02,
       -6.72005266e-02, -9.45669226e-03,  5.66317216e-02,  2.11554602e-01,
       -2.74622589e-02,  1.00099228e-01,  7.03393575e-03,  4.25531380e-02,
        4.70094830e-02, -1.03029683e-01,  5.88034838e-02, -5.67406453e-02,
       -2.86469832e-02, -5.46007492e-02, -3.57918106e-02, -4.15602140e-02,
       -7.03241527e-02,  3.93205397e-02, -6.03769645e-02,  7.21471086e-02,
        1.60594255e-01,  3.76343168e-02, -1.27753407e-01,  4.02488336e-02,
       -4.71676365e-02, -

In [376]:
'бузова' in navec_news

True

#### чтобы бороться с незнакомыми словами создадим словарь с 4граммами(по типу fasttext):
каждой 4грамме будет ставиться в соответствие среднее векторов от слов в которых встречается эта 4грамма

In [377]:
def get_ngrams(list_of_words, chars, embedding_model):
    ngrams = {}
    for word in list_of_words:
        vector_of_word = embedding_model[word]
        for i in range(len(word)-chars + 1):
            seq = word[i:i+chars]
            if seq in ngrams.keys():
                ngrams[seq].append(vector_of_word)
            else:
                ngrams[seq] = [vector_of_word]
      # усредним
    for seq in ngrams:
        ngrams[seq] = np.array(ngrams[seq]).mean(axis = 0)
    return ngrams

In [378]:
# получим n-граммы navec по 4 символа в n-грамме
ngrams_of_navec = get_ngrams(list_of_words = navec_news.vocab.words, chars = 4, embedding_model = navec_news)

In [379]:
ngrams_of_navec.keys()

dict_keys(['a-gp', '-gps', 'a-ha', 'a-on', '-one', 'aacu', 'acut', 'cute', 'aaro', 'aron', 'rons', 'onso', 'nson', 'aavs', 'avst', 'abar', 'abba', 'bbas', 'abbe', 'bbey', 'abbo', 'bbot', 'bott', 'abbv', 'bbvi', 'bvie', 'abby', 'bbyy', 'abcn', 'bcne', 'cnew', 'news', 'abda', 'bdal', 'dall', 'alla', 'llah', 'abde', 'bdel', 'abdu', 'bdul', 'dull', 'ulla', 'abel', 'bell', 'aben', 'bend', 'endb', 'ndbl', 'dbla', 'blat', 'latt', 'aber', 'berc', 'ercr', 'rcro', 'crom', 'romb', 'ombi', 'mbie', 'berd', 'erde', 'rdee', 'deen', 'able', 'abou', 'bout', 'abov', 'bove', 'abra', 'brah', 'raha', 'aham', 'bram', 'rams', 'abso', 'bsol', 'solu', 'olut', 'lute', 'utel', 'tely', 'abud', 'budh', 'udha', 'dhab', 'habi', 'abig', 'bigp', 'abus', 'buse', 'acad', 'cade', 'adem', 'demi', 'emia', 'emic', 'demy', 'acce', 'ccen', 'cent', 'entu', 'ntur', 'ture', 'ccep', 'cept', 'cces', 'cess', 'esso', 'ssor', 'sori', 'orie', 'ries', 'acci', 'ccid', 'cide', 'iden', 'dent', 'acco', 'ccor', 'cord', 'ordi', 'rdin', 'ding

#### Найдем mean и std для нормализации эмбеддингов, прежде чем обучать на них сеть. 


In [380]:
# из предобученных эмбеддингов
navec_news_vectors = np.array([navec_news[word] for word in navec_news.vocab.words])
navec_news_vectors.shape

(250002, 300)

In [381]:
# из ngrams
ngrams_vectors = np.array(list(ngrams_of_navec.values()))

In [382]:
emb_vectors = np.concatenate((ngrams_vectors, navec_news_vectors))

In [383]:
emb_mean = np.mean(emb_vectors, 0)
emb_std = np.std(emb_vectors, 0)

#### Average embedding.
Самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предложении(которые остались после токенизации и удаления коротких и стоп слов).

In [384]:
# функция, которая получает эмбеддинг заголовка, как среднее всех эмбеддингов слов в заголовке
def get_average_emb_of_title(title, tokenizer, chars = 4):
    """
    возвращает эмбеддинг заголовка, как среднее всех эмбеддингов слов в нем:
    title - заголовок, который нужно представить в векторном виде
    tokenizer - ваш токенизатор
    chars - кол-во символов при разбиении на n-граммы
    """
    # разбиваем на токены, приведя заголовок к нижнему регистру
    tokens = tokenizer.tokenize(title.lower())

    # фильтруем токены по длине, избавляемся от стопслов и чисел
    filtered_tokens = [token for token in tokens if (len(token) > 3) and 
                       token not in stopwords.words('russian') and 
                       token.isalpha()]
    # построим эмбединги
    embedding = []
    for token_word in filtered_tokens:

        if token_word in navec_news:  # для тех, которые есть в нашей модели
            embedding.append((navec_news[token_word] - emb_mean) / emb_std) # сразу же нормализуем

        else: # для тех слов, которых нет, строим его n-граммы
            ngrams= []
            # создаем n-граммы для незнокомого слова
            for i in range(len(token_word)-chars + 1):
                ngrams.append(token_word[i:i+chars])
      
            # для каждой n-граммы получаем ее представление из ngrams_of_navec
            for ngram in ngrams:
                if ngram in ngrams_of_navec.keys():
                    embedding.append(ngrams_of_navec[ngram])

    if len(embedding) == 0:
        return np.zeros((1, 300))
    elif len(embedding) == 1:
        return np.array(embedding).reshape(1, 300)
    else:
        return np.mean(np.array(embedding), 0).reshape(1, 300)

In [385]:
# для каждого заголовка получим его эмбеддинг
average_emb_train = []
average_emb_test = []

average_emb_make_predict = []

for i in range(len(train_data['title'])):
    average_emb_train.append(get_average_emb_of_title(train_data['title'][i], 
                                                      tokenizer = nltk.WordPunctTokenizer(), chars = 4))

for i in range(len(test_data['title'])):
    average_emb_test.append(get_average_emb_of_title(test_data['title'][i], 
                                                     tokenizer = nltk.WordPunctTokenizer(), chars = 4))

    
# для предсказаний
for i in range(len(make_predict['title'])):
    average_emb_make_predict.append(get_average_emb_of_title(make_predict['title'][i], 
                                                      tokenizer = nltk.WordPunctTokenizer(), chars = 4))
    
    
# создадим новую колонку, в которой будем хранить эмбеддинги предложений
train_data['emb'] = average_emb_train
test_data['emb'] = average_emb_test

#для предсказаний
make_predict['emb'] = average_emb_make_predict

In [386]:
print(len(make_predict.emb[0][0]))

300


In [387]:
np.array(make_predict['emb']).shape

(1000,)

In [388]:
# перемешаем индексы
n_train = len(train_data['emb'])
n_test = len(test_data['emb'])
n_pred = len(make_predict['emb'])
indexes = np.arange(n_train)
np.random.shuffle(indexes)

# и будем хранить в dict для удобства
train_dataset = {'features': np.array([train_data['emb'][i][0] for i in indexes]),  
                 'targets': np.array([train_data['is_fake'][i] for i in indexes])}

test_dataset = {'features': np.array([test_data['emb'][i][0] for i in range(n_test)]), 
                'targets': np.array([test_data['is_fake'][i] for i in range(n_test)])}

# для предиктов
make_predict_dataset = {'features': np.array([make_predict['emb'][i][0] for i in range(n_pred)]),  
                        'targets': np.array([make_predict['is_fake'][i] for i in range(n_pred)])}

In [389]:
make_predict_dataset['features'][999].shape, make_predict_dataset['targets'][999].shape

((300,), ())

In [390]:
type(make_predict_dataset['features'][0])

numpy.ndarray

In [391]:
# Создадим класс MyDataset, который наследуется от класса torch.utils.data.Dataset,
# чтобы отправить данные в DataLoader, чтобы отправить в batch_generator
class MyDataset(Dataset):
  
    def __init__(self, data: dict, features: str, targets: str):

        self.data = data
        self.features = features
        self.targets = targets

        self.title = torch.Tensor(data[features])
        self.is_fake = torch.LongTensor(data[targets])

    def __len__(self):
        return self.data[self.features].shape[0]
  
    def __getitem__(self, index):
        return (self.title[index], self.is_fake[index])

In [392]:
# приведем данные к tensor формату, чтобы отправить в dataloader
train_load = MyDataset(train_dataset, 'features', 'targets')

test_load = MyDataset(test_dataset, 'features', 'targets')

#для предиктов
make_predict_load = MyDataset(make_predict_dataset, 'features', 'targets')

начнем строить модель 

##### функции-обучатели

In [47]:
def trainer(count_of_epoch, batch_size, dataset, model, loss_function, optimizer, lr = 0.001):
    """
    trainer итерируется по кол-ву эпох и вызывает функцию train_epoch
    count_of_epoch - кол-во эпох
    batch_size - размер батча
    dataset - данные для обучения
    model - модель нейронной сети
    loss_function - функция потерь
    optimizer - оптимизатор
    lr - скорость обучения, по умолчанию 0.001
    """
    optima = optimizer(model.parameters(), lr=lr)
    
    iterations = tqdm(range(count_of_epoch), desc='epoch')
    iterations.set_postfix({'train epoch loss': np.nan})
    for it in iterations:
        batch_generator = tqdm(
            torch.utils.data.DataLoader(dataset=dataset, 
                                        batch_size=batch_size, 
                                        shuffle=True, pin_memory=True), 
            leave=False, total=len(dataset)//batch_size+(len(dataset)%batch_size>0))
        
        epoch_loss = train_epoch(train_generator=batch_generator, 
                    model=model, 
                    loss_function=loss_function, 
                    optimizer=optima)
        
        iterations.set_postfix({'train epoch loss': epoch_loss})

In [117]:
def train_epoch(train_generator, model, loss_function, optimizer):
    """
    внутри train_epoch итерируемся по батчам внутри батчгенератора
    train_generator - батчгенератора
    model - модель нейронной сети
    loss_function - функция потерь
    optimizer - оптимизатор
    """
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x, batch_of_y, optimizer, loss_function)
            
        epoch_loss += batch_loss*len(batch_of_x)
        total += len(batch_of_x)
    
    return epoch_loss/total

In [118]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    """
    в train_on_batch обучаемся на одном батче
    model - модель нейронной сети
    x_batch - фичи
    y_batch - таргеты(метки классов)
    optimizer - оптимизатор
    loss_function - функция потерь
    """
    model.train()
    optimizer.zero_grad()
    
    output = model(x_batch.to(device))
    
    loss = loss_function(output, y_batch.to(device))
    loss.backward()

    optimizer.step()
    return loss.cpu().item()

##### модель сети

In [119]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

In [335]:
# AVERAGE MODEL
D_in = 300   #размерность входа
H = 10      #размерность скрытых слоев
D_out = 2    #размерность выхода
model =  nn.Sequential(
    nn.Linear(D_in, H),  
    nn.Tanh(),           
    nn.Linear(H, H),  
    nn.Tanh(),
    nn.Linear(H, D_out),
    nn.Tanh()) 
_ = model.to(device)

качество до обучения

In [336]:
batch_generator = torch.utils.data.DataLoader(dataset=test_load, 
                                              batch_size=32, 
                                              pin_memory=True)
            
pred = []
real = []
model.eval()
for it, (x_batch, y_batch) in enumerate(batch_generator):
    x_batch = x_batch.to(device)
    with torch.no_grad():
        output = model(x_batch)

    pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
    real.extend(y_batch.cpu().numpy().tolist())

print(classification_report(real, pred, zero_division = 0))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67       574
           1       0.00      0.00      0.00       578

    accuracy                           0.50      1152
   macro avg       0.25      0.50      0.33      1152
weighted avg       0.25      0.50      0.33      1152



обучение

In [337]:
trainer(count_of_epoch = 10,  
        batch_size = 16, 
        dataset = train_load,
        model = model, 
        loss_function = loss_function,
        optimizer = optimizer,
        lr = 0.001)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

...

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

качество после обучения

In [338]:
batch_generator = torch.utils.data.DataLoader(dataset=test_load, 
                                              batch_size=32, 
                                              pin_memory=True)

pred = []
real = []
model.eval()
for it, (x_batch, y_batch) in enumerate(batch_generator):
    x_batch = x_batch.to(device)
    with torch.no_grad():
        output = model(x_batch)
        
    pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
    real.extend(y_batch.cpu().numpy().tolist())

print(classification_report(real, pred))

              precision    recall  f1-score   support

           0       0.85      0.78      0.82       574
           1       0.80      0.87      0.83       578

    accuracy                           0.82      1152
   macro avg       0.83      0.82      0.82      1152
weighted avg       0.83      0.82      0.82      1152



Получили хорошее качество, причем обучение занимает около 5сек, места в памяти также много не тратится: google collab показывал 2.88гб ОЗУ

Вывод: лучше использовать простые модели, которые, незатратно и быстро дают хорошие результаты и обладают лучшей обобщающей способностью.

#### сделаем предсказания на файле test.tsv

In [393]:
batch_generator = torch.utils.data.DataLoader(dataset=make_predict_load, 
                                              batch_size=32, 
                                              pin_memory=True)

pred = []
real = []
model.eval()
for it, (x_batch, y_batch) in enumerate(batch_generator):
    x_batch = x_batch.to(device)
    with torch.no_grad():
        output = model(x_batch)
        
    pred.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
    
predicted_labels = pred

In [394]:
make_predict.is_fake = predicted_labels
make_predict.drop(columns = ['emb'], inplace = True)

In [398]:
make_predict.to_csv('predictions.tsv', sep = '\t', index = False)